In [87]:
#initialization
import matplotlib.pyplot as plt
import numpy as np
import math
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Statevector
from qiskit.primitives import Sampler
from qiskit.visualization import plot_histogram, plot_bloch_vector, plot_state_qsphere
from qiskit.quantum_info import Operator

In [88]:
# visualize eigenstate
def to_bloch_vector(ucirc, statevector):
    new_state = statevector.evolve(ucirc)
    bloch_vector = [
    new_state.expectation_value(Operator([[0, 1], [1, 0]])).real, # X
    new_state.expectation_value(Operator([[0, -1j], [1j, 0]])).real, # Y
    new_state.expectation_value(Operator([[1, 0], [0, -1]])).real # Z
    ]
    return bloch_vector
# visualize eigenvalue
def plot_complex_nmb(phasors):
    # Create a new figure
    plt.figure(figsize=(4, 4))
    # plot each phasor as a vector
    for phasor in phasors:
        real = np.real(phasor)
        imag = np.imag(phasor)
        plt.quiver(0, 0, real, imag, angles='xy', scale_units='xy', scale=1, color='r')
        plt.text(real * 1.1, imag * 1.1, f'{phasor:.2f}', fontsize=12)
    # set up the plot axes
    plt.xlim(-2, 2) # Adjust these limits based on your phasors
    plt.ylim(-2, 2)
    plt.axhline(0, color='black',linewidth=0.5)
    plt.axvline(0, color='black',linewidth=0.5)
    plt.xlabel('Real')
    plt.ylabel('Imaginary')
    plt.grid(True)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title('Phasor Diagram')
    # show the plot
    plt.show()    

In [181]:
phi = np.pi/3
theta = 11/12
n = 4
# create quantum circuit
qc = QuantumCircuit(n+1, n)
# prepare eigenstate
# qc.x(n)
qc.h(n)
# step 1: prepare ancilla qubit
for k in range(n):
    qc.h(k)
    qc.s(k)
# step 2: controlled unitary operations
for k in range(n):
    repetitions = 2**(k)
    qc.crx(repetitions*phi, k , n)
    # step 3: apply Hadamard to control qubit
    qc.h(k)
    qc.barrier()
# step 4: measurement
for k in range(n):
    qc.measure(k,k)
qc.draw(output='mpl').savefig(f'kqpe_{n}.pdf')
sampler = Sampler()
job = sampler.run(circuits=qc, shots=5000)
result = job.result()

/var/folders/sn/4ym_qb8x0cs_f961d78kvvkc0000gn/T/ipykernel_49922/2352764313.py:24: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


In [182]:
# get the result counts and convert quasi-probabilities to probabilities
counts = result.quasi_dists[0].binary_probabilities()
# print result
print("Measuremet result:", result.quasi_dists[0])
# plot the result as a histogram
plt.rc('font', family='serif', serif='Times New Roman')
plot_histogram(counts, figsize=(8,4.5)).savefig(f'hist_kqpe_{n}.pdf')

Measuremet result: {0: np.float64(0.0378), 1: np.float64(0.0112), 2: np.float64(0.0022), 3: np.float64(0.0012), 4: np.float64(0.0034), 5: np.float64(0.001), 7: np.float64(0.0002), 8: np.float64(0.608), 9: np.float64(0.213), 10: np.float64(0.0442), 11: np.float64(0.017), 12: np.float64(0.0418), 13: np.float64(0.0158), 14: np.float64(0.002), 15: np.float64(0.0012)}


In [183]:
counts

{'0000': np.float64(0.0378),
 '0001': np.float64(0.0112),
 '0010': np.float64(0.0022),
 '0011': np.float64(0.0012),
 '0100': np.float64(0.0034),
 '0101': np.float64(0.001),
 '0111': np.float64(0.0002),
 '1000': np.float64(0.608),
 '1001': np.float64(0.213),
 '1010': np.float64(0.0442),
 '1011': np.float64(0.017),
 '1100': np.float64(0.0418),
 '1101': np.float64(0.0158),
 '1110': np.float64(0.002),
 '1111': np.float64(0.0012)}

In [164]:
def decimal_to_binary(decimal, precision=5):
    """
    Convert a decimal number in [0, 1) to binary representation.
    
    Parameters:
        decimal (float): The decimal number to convert (must be between 0 and 1).
        precision (int): The number of binary places to compute.
        
    Returns:
        str: The binary representation of the decimal number.
    """
    if not (0 <= decimal < 1):
        raise ValueError("Input should be a decimal between 0 and 1.")
    
    binary = "0."
    for _ in range(precision):
        decimal *= 2
        bit = int(decimal)
        if bit == 1:
            decimal -= bit
            binary += "1"
        else:
            binary += "0"
    return binary

def binary_to_decimal(binary_str):
    """
    Convert a binary fractional string (e.g., '0.101') to a decimal number.
    
    Parameters:
        binary_str (str): The binary string to convert (must start with '0.').
        
    Returns:
        float: The decimal representation of the binary fraction.
    """
    if not binary_str.startswith("0.") or any(bit not in '01' for bit in binary_str[2:]):
        raise ValueError("Input should be a binary string starting with '0.' and containing only '0' and '1' after the decimal point.")
    
    decimal = 0.0
    for i, bit in enumerate(binary_str[2:], start=1):
        decimal += int(bit) * 2**-i
    
    return decimal

In [184]:
decimal_to_binary(11/12)

'0.11101'

In [166]:
qc_0 = QuantumCircuit(3,3)
qc_0.x(0)
for i in range(3):
    qc_0.measure(i,i)

sampler = Sampler()
job = sampler.run(circuits=qc_0, shots=5000)
result = job.result()

/var/folders/sn/4ym_qb8x0cs_f961d78kvvkc0000gn/T/ipykernel_49922/2587719164.py:6: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


In [167]:
counts = result.quasi_dists[0].binary_probabilities()
print(counts)

{'001': np.float64(1.0)}
